# Imports and Data Loading

In [ ]:
import json
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import box
import random
import geopy.distance
import pandas as pd
import numpy as np
from geovoronoi import voronoi_regions_from_coords, points_to_coords
import networkx as nx


In [ ]:
path = ""
puglia_prov = gpd.read_file(path+"/puglia_prov.geojson")
puglia_mun = gpd.read_file(path+"/puglia_mun.geojson")
uliveti = gpd.read_file(path+"/uliveti.geojson")

In [ ]:
uliveti1 = uliveti.copy()
uliveti1 = uliveti1.to_crs(epsg=3395)
uliveti1['geometry'] = uliveti1.centroid

In [ ]:
area = puglia_prov[puglia_prov['DEN_PROV'] == "Lecce"]
#area big if in Lecce, Brindisi or Taranto
area_big = puglia_prov[puglia_prov['DEN_PROV'].isin(["Lecce","Brindisi","Taranto"])]
#merge
area = area.to_crs(epsg=3395)    # convert to World Mercator CRS
area_big = area_big.to_crs(epsg=3395)
area_shape = area.iloc[0].geometry
#merge together the polygon of the three provinces in area big
area_shape_big = area_big.unary_union 
uliveti1 = uliveti1[uliveti1.within(area_shape)]

# Constructions of Voronoi regions

In [ ]:
coords = points_to_coords(uliveti1.geometry)
region_polys, region_pts = voronoi_regions_from_coords(coords, area_shape.buffer(0.1))

# Creation of edgelist
Two nodes share an edge if the two corresponding Voronoi regions share a common border

In [ ]:

# dictionary where for each voronoi region id there is the number of the corresponding neighboring regions 
dict_edgelist = {}
j=0
for key,value in region_polys.items():
    lista = []
    for key1,value1 in region_polys.items():
        if value1.touches(value):
            lista.append(key1)
    dict_edgelist[key] = lista
    j+=1
    print(j)

# Creation of the graph object

In [ ]:
dict_degree = {int(k): v for k, v in dict_edgelist.items()}
G = nx.Graph(dict_degree)